In [158]:
from bs4 import BeautifulSoup as bs
import requests
import pymongo
from splinter import Browser
import pandas as pd
import re

In [100]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [101]:
# Define database and collection
db = client.NavySchedule_db
collection = db.items

In [102]:
headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36'}
res = requests.get('https://navysports.com/schedule.aspx?path=wlax', headers=headers)
soup = bs(res.content, 'html.parser')
soup.find_all('span', class_='sidearm-schedule-game-opponent-name')

[<span class="sidearm-schedule-game-opponent-name">
 <a aria-label="#16 Virginia on February 9 1:00 p.m." href="http://virginiasports.com/" target="_blank">#16 Virginia</a>
 </span>, <span class="sidearm-schedule-game-opponent-name">
 <a aria-label="Saint Joseph's on February 17 1:00 p.m." href="http://www.sjuhawks.com/" target="_blank">Saint Joseph's</a>
 </span>, <span class="sidearm-schedule-game-opponent-name">
 <a aria-label="#19 Towson on February 23 3:00 p.m." href="http://www.towsontigers.com/" target="_blank">#19 Towson</a>
 </span>, <span class="sidearm-schedule-game-opponent-name">
 <a aria-label="Villanova on February 27 3:00 p.m." href="http://www.villanova.com/" target="_blank">Villanova</a>
 </span>, <span class="sidearm-schedule-game-opponent-name">
 <a aria-label="#11 Florida on March 2 11:00 a.m." href="http://www.floridagators.com/" target="_blank">#11 Florida</a>
 </span>, <span class="sidearm-schedule-game-opponent-name">
 <a aria-label="#1 Boston College on March 

In [103]:
# Examine the results, then determine element that contains sought info
# results are returned as an iterable list
results = soup.find_all('span', class_='sidearm-schedule-game-opponent-name')

opponent = []

# Loop over td elements
for result in results:
            stuff = (result.a.text)
            clean_stuff= re.sub("\d", "",stuff)
            clean_stuff= re.sub("# ", "",clean_stuff)
            opponent.append(clean_stuff)

print(opponent)


['Virginia', "Saint Joseph's", 'Towson', 'Villanova', 'Florida', 'Boston College', 'Brown', 'Colgate', 'Bucknell', 'Lehigh', 'Holy Cross', 'Loyola', 'Georgetown', 'Army West Point', 'American', 'Lafayette', 'Boston University', 'Army West Point', 'Loyola', 'High Point']


In [199]:
results2 = soup.find_all('span', class_='sidearm-schedule-game-conference-vs flex flex-inline')                                
  
home_away = []

# Loop over elements
for result2 in results2:
    if (result2.text):
            stuff = (result2.text)
            clean_stuff= re.sub("\n", "",stuff)
            home_away.append(clean_stuff)
            
print(home_away)


['at', 'vs', 'vs', 'at', 'vs', 'vs', 'vs', 'at', 'vs', 'at', 'vs', 'at', 'at', 'vs', 'vs', 'vs', 'at', 'vs', 'at', 'vs']


In [200]:
from datetime import datetime

results3 = soup.find_all('div', class_='sidearm-schedule-game-opponent-date flex-item-1')

period = []
day = []


# Loop over elements
for result3 in results3:
            stuff = ((result3.span.text)[0:3])
            clean_stuff = datetime.strptime(stuff,'%b')
            clean_stuff = clean_stuff.strftime('%m')
            period.append(clean_stuff)
            


            date_day = ((result3.span.text)[4:6])
            date_day = re.sub(" ", "",date_day)
            if len(date_day) == 1:
                date_day = "0" + date_day
            else:
                date_day = date_day
            day.append(date_day)   
            
print(period)          
print(day)   

        
#apply formula to get period number for left 3
#apply formula if len = 11 then take 5th character and add 0
#else take 5th and 6th character

['02', '02', '02', '02', '03', '03', '03', '03', '03', '03', '03', '04', '04', '04', '04', '04', '04', '05', '05', '05']
['09', '17', '23', '27', '02', '06', '09', '16', '23', '27', '30', '06', '10', '13', '17', '20', '25', '02', '04', '10']


In [202]:
import numpy as np
url_list = pd.DataFrame(np.column_stack([opponent,home_away,period,day]), columns=['key','Home_Away','Period','Day'])
url_list

,key,Home_Away,Period,Day
0,Virginia,at,02,09
1,Saint Joseph's,vs,02,17
2,Towson,vs,02,23
3,Villanova,at,02,27
4,Florida,vs,03,02
5,Boston College,vs,03,06
6,Brown,vs,03,09
7,Colgate,at,03,16
8,Bucknell,vs,03,23
9,Lehigh,at,03,27


In [203]:
# The path to our CSV file
file = "SystemSchool.csv"

# Read our Kickstarter data into pandas
df = pd.read_csv(file)
df.head()

,key,CONFERENCE,SystemName
0,Maryland,Big Ten,maryland
1,Boston College,ACC,boston-college
2,North Carolina,ACC,north-carolina
3,Northwestern,Big Ten,northwestern
4,Syracuse,ACC,syracuse


In [204]:
new_df = url_list.set_index('key').join(df.set_index('key'))
new_df

,Home_Away,Period,Day,CONFERENCE,SystemName
key,,,,,
American,vs,04,17,Patriot,american
Army West Point,vs,04,13,Patriot,army
Army West Point,vs,05,02,Patriot,army
Boston College,vs,03,06,ACC,boston-college
Boston University,at,04,25,Patriot,boston-u
Brown,vs,03,09,Ivy League,brown
Bucknell,vs,03,23,Patriot,bucknell
Colgate,at,03,16,Patriot,colgate
Florida,vs,03,02,AAC,florida


In [232]:
ncaa_url_base = 'https://www.ncaa.com/game/lacrosse-women/d1/2019/'

url_dict = []
    
for index, row in new_df.iterrows():
    if row['Home_Away'] == 'at':
        url = ncaa_url_base + row['Period'] + '/' + row['Day'] + '/navy-' +  row['SystemName'] + '/play-by-play'
        url_dict.append(url)
    else:
        url = ncaa_url_base + row['Period'] + '/' + row['Day'] + '/' +  row['SystemName'] + '-navy/play-by-play'
        url_dict.append(url)
print(list(url_dict))


['https://www.ncaa.com/game/lacrosse-women/d1/2019/04/17/american-navy/play-by-play', 'https://www.ncaa.com/game/lacrosse-women/d1/2019/04/13/army-navy/play-by-play', 'https://www.ncaa.com/game/lacrosse-women/d1/2019/05/02/army-navy/play-by-play', 'https://www.ncaa.com/game/lacrosse-women/d1/2019/03/06/boston-college-navy/play-by-play', 'https://www.ncaa.com/game/lacrosse-women/d1/2019/04/25/navy-boston-u/play-by-play', 'https://www.ncaa.com/game/lacrosse-women/d1/2019/03/09/brown-navy/play-by-play', 'https://www.ncaa.com/game/lacrosse-women/d1/2019/03/23/bucknell-navy/play-by-play', 'https://www.ncaa.com/game/lacrosse-women/d1/2019/03/16/navy-colgate/play-by-play', 'https://www.ncaa.com/game/lacrosse-women/d1/2019/03/02/florida-navy/play-by-play', 'https://www.ncaa.com/game/lacrosse-women/d1/2019/04/10/navy-georgetown/play-by-play', 'https://www.ncaa.com/game/lacrosse-women/d1/2019/05/10/high-point-navy/play-by-play', 'https://www.ncaa.com/game/lacrosse-women/d1/2019/03/30/holy-cross-